# Práctica 4
<br>

__Alumnos:__
* __Frederick Ernesto Borges Noronha__
* __Victor Manuel Cavero Gracia__


## PARTE 1

Implementa una función que, dados dos identificadores de ítems y una distancia
máxima, devuelva los ítems comunes que están como máximo a esa distancia de los
dos ítems iniciales.

Como Wikidata es muy grande vamos a limitarnos a buscar entidades comunes a
distancia máxima 3 usando sólo el siguiente subconjunto de propiedades:
`instance of (P31)` , `subclass of (P279)`, `has part (P527)`,
`instrument (P1303)`, `genre (P136)`

Si no hay entidades comunes a esa distancia diremos que los ítems no están
relacionados.

Calcula las relaciones entre las siguientes entidades dos a dos y explica en lenguaje
natural el tipo de relación encontrada:
`piano (Q5994)`, `electronic keyboard (Q1343007)`,
`String synthesizer (Q2355465)`, `Hans Zimmer (Q76364)`,
`Pirates of the Caribbean: The Curse of the Black Pearl (Q46717)`,
`The Lion King (Q36479)`, `Toy Story (Q171048)`, `Iron Man (Q192724)`.

- Consejos:
  - Obtener la información de una entidad Wikidata con get_entity_dict_from_api es
    una operación lenta. Te recomendamos que crees una cache de entidades (map
    entidad_id -> entidad) que evite pedir una y otra vez las mismas entidades a
    Wikidata.
  - Te recomendamos crear una función get_claims(item_id) que devuelva las
    relaciones que salen a partir de una entidad en formato (id_propiedad, id_valor).
    Sólo debes considerar los truthy_claims que tengan como valor un item de
    Wikidata (entidades que empiezan por ‘Q’). Recuerda que también hemos
    limitado las propiedades que debes considerar.
  - Es importante tener en cuenta que, cuando expandimos una onda, la intersección
    se puede producir con cualquiera de las entidades de la otra onda (y no sólo con
    las de la frontera).


In [1]:
from qwikidata.entity import WikidataItem, WikidataProperty, WikidataLexeme 
from qwikidata.linked_data_interface import get_entity_dict_from_api


def items_comunes(item_id1, item_id2, distancia_maxima=3):
    dict_cache = dict() # Caché de la forma entidad_id -> entidad
    items_comunes = [] # Lista con (entidad_id,distancia)
    
    dist1_id1 = get_claims(item_id1, dict_cache) # Distancia de 1 a item_id1
    dist1_id2 = get_claims(item_id2, dict_cache) # Distancia de 1 a item_id2
    dic_dist2_id1 = dict() # Distancia de 2 con Diccionario < (id_propiedad, id_valor), [(id_propiedad, id_valor)] >
    dic_dist2_id2 = dict() # Distancia de 2 con Diccionario < (id_propiedad, id_valor), [(id_propiedad, id_valor)] >
    
    for pair in dist1_id1:
        dic_dist2_id1[pair] = get_claims(pair[1], dict_cache)

    for pair in dist1_id2:
        dic_dist2_id2[pair] = get_claims(pair[1], dict_cache)
    
    for pair_id1 in dist1_id1:
        for pair_id2 in dist1_id2:
            if(pair_id1[1] == pair_id2[1]):
                items_comunes.append((pair_id2[1],2))
            for pair2_id1 in dic_dist2_id1[pair_id1]:
                if(pair2_id1[1] == pair_id2[1]):
                    items_comunes.append((pair_id2[1],3))
            for pair2_id2 in dic_dist2_id2[pair_id2]:
                if(pair2_id2[1] == pair_id1[1]):
                    items_comunes.append((pair2_id2[1],3))
    
    cadena = '--- ENTIDAD: {} , NOMBRE {} Y DISTANCIA: {} '
    
    for item in items_comunes:
        if(item[1] <= distancia_maxima):
            if item[0] in dict_cache : print(cadena.format(item[0], dict_cache[item[0]].get_label(), item[1]))
            else :
                item_dict = get_entity_dict_from_api(item[0])
                entidad = WikidataItem(item_dict)
                print(cadena.format(item[0], entidad.get_label(), item[1]))
    
def get_claims(item_id, dict_cache):
    # truthly_claims que empiezen por 'Q' id_valor y id_propiedad sea : P31, P279, P527, P1303 o P136
    
    tuple_list = []
    propiedades = ['P31','P279','P527','P1303','P136']

    if item_id in dict_cache:
        entidad = dict_cache[item_id]
    else:    
        item_dict = get_entity_dict_from_api(item_id)
        entidad = WikidataItem(item_dict)
        dict_cache[item_id] = entidad
    
    for x in propiedades:
        claim_group = entidad.get_truthy_claim_group(x)
        for y in claim_group:
            tuple_list.append((x,y.mainsnak.datavalue.value['id']))
            
    return tuple_list # (id_propiedad, id_valor)
            

Calcular las relaciones entre las siguientes entidades dos a dos y explica en lenguaje
natural el tipo de relación encontrada:

In [2]:
items_comunes('Q5994','Q1343007')

--- ENTIDAD: Q34379 , NOMBRE musical instrument Y DISTANCIA: 3 
--- ENTIDAD: Q52954 , NOMBRE keyboard instrument Y DISTANCIA: 2 


In [3]:
items_comunes('Q2355465','Q76364')

--- ENTIDAD: Q1327500 , NOMBRE electronic musical instrument Y DISTANCIA: 3 


In [4]:
items_comunes('Q46717','Q36479')

--- ENTIDAD: Q11424 , NOMBRE film Y DISTANCIA: 3 
--- ENTIDAD: Q11424 , NOMBRE film Y DISTANCIA: 3 
--- ENTIDAD: Q11424 , NOMBRE film Y DISTANCIA: 3 
--- ENTIDAD: Q11424 , NOMBRE film Y DISTANCIA: 3 


In [5]:
items_comunes('Q171048','Q192724')

--- ENTIDAD: Q11424 , NOMBRE film Y DISTANCIA: 3 


## PARTE 2

Implementa una versión avanzada del algoritmo visto en clase que no sólo devuelva
las entidades comunes sino también los caminos que conectan cada una de las
entidades originales con esas entidades comunes. En este caso, la función sólo debe
devolver los caminos de longitud mínima. Si no hay caminos de longitud menor o igual
a 5 diremos que las entidades no están relacionadas.

- Consejos:
    - La intersección de las dos ondas puede contener varias entidades. Debes
        considerar todas ellas a la hora de calcular las posibles soluciones. Además, cada
        una de las entidades comunes puede dar lugar a varias soluciones si hay distintos
        caminos que conectan las entidades iniciales con esa entidad común.
    - De todas las soluciones posibles sólo debes devolver las de longitud mínima,
        entendiendo que la longitud de la solución es la suma de las longitudes de los
        caminos desde las entidades iniciales hasta la entidad común.
    - Hay muchas formas de implementar este algoritmo. Una de ellas consiste en
        almacenar todos los caminos de cada una de las ondas e ir expandiéndolos
        alternativamente hasta que las ondas intersequen. Para calcular la intersección
        deberás almacenar también los ítems de cada una de las ondas.
    - Ten en cuenta que para calcular todas las posibles soluciones de longitud <= n
        debes expandir cada una de las ondas n veces (puede que la relación vaya del
        primer ítem hasta el segundo).

In [6]:
from qwikidata.entity import WikidataItem, WikidataProperty, WikidataLexeme 
from qwikidata.linked_data_interface import get_entity_dict_from_api


def camino_minimo(item_id1, item_id2, distancia_maxima=3):
    dict_cache = dict() # Caché de la forma entidad_id -> entidad
    items_comunes = [] # Lista con (entidad_id,distancia)
    
    dist1_id1 = get_claims(item_id1, dict_cache) # Distancia de 1 a item_id1
    dist1_id2 = get_claims(item_id2, dict_cache) # Distancia de 1 a item_id2
    dic_dist2_id1 = dict() # Distancia de 2 con Diccionario < (id_propiedad, id_valor), [(id_propiedad, id_valor)] >
    dic_dist2_id2 = dict() # Distancia de 2 con Diccionario < (id_propiedad, id_valor), [(id_propiedad, id_valor)] >
    
    for pair in dist1_id1:
        dic_dist2_id1[pair] = get_claims(pair[1], dict_cache)

    for pair in dist1_id2:
        dic_dist2_id2[pair] = get_claims(pair[1], dict_cache)
    
    for pair_id1 in dist1_id1:
        for pair_id2 in dist1_id2:
            if(pair_id1[1] == pair_id2[1]):
                items_comunes.append((pair_id2[1],2))
            for pair2_id1 in dic_dist2_id1[pair_id1]:
                if(pair2_id1[1] == pair_id2[1]):
                    items_comunes.append((pair_id2[1],3))
            for pair2_id2 in dic_dist2_id2[pair_id2]:
                if(pair2_id2[1] == pair_id1[1]):
                    items_comunes.append((pair2_id2[1],3))
    
    cadena = '--- ENTIDAD: {} , NOMBRE {} Y DISTANCIA: {} '
    
    for item in items_comunes:
        if(item[1] <= distancia_maxima):
            if item[0] in dict_cache : print(cadena.format(item[0], dict_cache[item[0]].get_label(), item[1]))
            else :
                item_dict = get_entity_dict_from_api(item[0])
                entidad = WikidataItem(item_dict)
                print(cadena.format(item[0], entidad.get_label(), item[1]))
    
            

Comparación de las entidades del apartado anterior dos a dos mostrando los
caminos de longitud mínima. Considerando las propiedades indicadas
previamente